In [26]:
# Import Packages
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
pd.set_option('display.max_colwidth', 100)
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
from gensim.models import KeyedVectors
import gensim.downloader
from collections import deque

In [27]:
# Load pretrained word2vec model from gensim
google_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

In [28]:
# load csv from zip file
main_data = pd.read_csv('/Users/andrewsimon/Desktop/IMDBDataset.csv.zip')
main_data

,review,sentiment
0,One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....,positive
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...,positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei off...",positive
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, ...",positive
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...",negative
49997,"I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...",negative
49998,I'm going to have to disagree with the previous comment and side with Maltin on this one. This i...,negative


In [29]:
# Map sentiment via one hot encoding
main_data['sentiment'] = main_data['sentiment'].map({'positive':1, 'negative': 0})
main_data

,review,sentiment
0,One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....,1
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...,1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",1
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...,0
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei off...",1
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, ...",1
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...",0
49997,"I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...",0
49998,I'm going to have to disagree with the previous comment and side with Maltin on this one. This i...,0


In [30]:
# Preprocess data for W2v
main_data['review'] = main_data['review'].apply(lambda x: gensim.utils.simple_preprocess(x))
main_data.head()

,review,sentiment
0,"[one, of, the, other, reviewers, has, mentioned, that, after, watching, just, oz, episode, you, ...",1
1,"[wonderful, little, production, br, br, the, filming, technique, is, very, unassuming, very, old...",1
2,"[thought, this, was, wonderful, way, to, spend, time, on, too, hot, summer, weekend, sitting, in...",1
3,"[basically, there, family, where, little, boy, jake, thinks, there, zombie, in, his, closet, his...",0
4,"[petter, mattei, love, in, the, time, of, money, is, visually, stunning, film, to, watch, mr, ma...",1


In [31]:
main_data = main_data.sample(n=500, random_state=1516)
main_data = main_data.reset_index()
main_data

,index,review,sentiment
0,13974,"[best, years, of, our, lives, is, film, that, slipped, under, my, radar, for, years, had, heard,...",1
1,25449,"[mitchell, leisen, fifth, feature, as, director, and, he, shows, his, versatility, by, directing...",1
2,11079,"[if, you, like, movies, about, creepy, towns, hotels, houses, states, ala, the, eagles, hotel, c...",0
3,28464,"[after, receiving, dvd, of, this, with, sunday, newspaper, hoped, that, it, was, not, the, usual...",0
4,44821,"[victor, buono, as, the, devil, surely, somebody, must, have, been, drunk, when, that, casting, ...",0
...,...,...,...
495,32075,"[but, you, can, see, it, from, here, br, br, definitely, don, understand, why, anyone, would, re...",0
496,40300,"[thought, anywhere, but, here, was, good, movie, it, stars, two, wonderful, actresses, susan, sa...",1
497,47265,"[this, is, very, entertaining, flick, considering, the, budget, and, its, length, the, storyline...",1
498,12360,"[all, could, think, while, watching, this, movie, was, will, it, ever, end, it, was, unbearably,...",0


In [32]:

# Append vectorizations from pre trained model

for i in range(len(main_data['review'])):
    sub_list = []
    for j in main_data['review'][i]:
        try:
            sub_list.append(google_vectors[j])
        except KeyError:
            sub_list.append(np.zeros(300,))
    main_data['review'][i] = sub_list


/var/folders/8f/mjj28lvs5fb1m_c_tvmtckl80000gn/T/ipykernel_31124/2852260835.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['review'][i] = sub_list


In [33]:
# Separate into training and testing data
X_train, X_test, y_train, y_test = train_test_split(main_data['review'], main_data['sentiment'], test_size=0.2, random_state=1516)

In [34]:
# Pad data to remove ragged arrays
training_padded = pad_sequences(X_train, maxlen=120, truncating='post')

In [35]:

testing_padded = pad_sequences(X_test, maxlen=120, truncating='post')

In [36]:
# Cast to numpy arrays
X_train = X_train.to_numpy()

In [37]:
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [38]:
# Create model, evaluate

batch = 32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Recall', 'AUC', 'Precision', 'FalseNegatives', 'FalsePositives'])
    
num_epochs = 10
model.fit(training_padded, y_train, epochs=num_epochs,batch_size=batch, validation_data=(testing_padded, y_test))

Epoch 1/10
13/13 [==============================] - 1s 17ms/step - loss: 0.6906 - accuracy: 0.5325 - recall: 0.1154 - auc: 0.5081 - precision: 0.4468 - false_negatives: 161.0000 - false_positives: 26.0000 - val_loss: 0.6926 - val_accuracy: 0.5200 - val_recall: 0.0000e+00 - val_auc: 0.5603 - val_precision: 0.0000e+00 - val_false_negatives: 48.0000 - val_false_positives: 0.0000e+00
Epoch 2/10
13/13 [==============================] - 0s 9ms/step - loss: 0.6516 - accuracy: 0.6075 - recall: 0.1813 - auc: 0.8370 - precision: 0.8049 - false_negatives: 149.0000 - false_positives: 8.0000 - val_loss: 0.6958 - val_accuracy: 0.5400 - val_recall: 0.5833 - val_auc: 0.5286 - val_precision: 0.5185 - val_false_negatives: 20.0000 - val_false_positives: 26.0000
Epoch 3/10
13/13 [==============================] - 0s 6ms/step - loss: 0.6010 - accuracy: 0.7725 - recall: 0.5495 - auc: 0.9313 - precision: 0.9174 - false_negatives: 82.0000 - false_positives: 9.0000 - val_loss: 0.6984 - val_accuracy: 0.5100 - v

In [40]:
training_padded_w2v = training_padded
testing_padded_w2v = testing_padded

%store training_padded_w2v
%store testing_padded_w2v

Stored 'training_padded_w2v' (ndarray)
Stored 'testing_padded_w2v' (ndarray)
